In [1]:
import transformers
import sys
sys.path.append('../')
sys.path.append('./')
sys.path.append('/home/amber/multitask_RNA/rna_self_train')

from dnabert_datastruct import mask_tokens
from dnabert_datastruct import DNATokenizer
from torch.utils.data import DataLoader
import rna_model
import importlib
import numpy as np
import torch
from transformers import RobertaConfig, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments

In [2]:
class arg():
    def __init__(self,prb):
        self.mlm_probability = prb

class rnabert_maskwrapper():
    def __init__(self,tokenizer,prob_arg) -> None:
        self.tokenizer = tokenizer
        self.prb = prob_arg
    def __call__(self, batch_entry):
        batch_entry = torch.from_numpy(np.array(batch_entry))
        input,label = mask_tokens(batch_entry,self.tokenizer,arg(self.prb))
        return{'input_ids':input,'labels':label}

In [3]:
importlib.reload(rna_model)
tokenizer = DNATokenizer.from_pretrained('dna6')
train_data = rna_model.rna_kmer('../data/pre-train/510/rna_seq.h5','train',6,tokenizer)
valid_data = rna_model.rna_kmer('../data/pre-train/510/rna_seq.h5','valid',6,tokenizer)
data_collator = rnabert_maskwrapper(tokenizer,0.05)

<class 'dnabert_datastruct.DNATokenizer'>


In [4]:
test_seq = valid_data.__getitem__(1)

In [5]:
data_collator([test_seq])

tensor([[False, False,  True, False, False, False,  True, False,  True, False,
         False, False, False, False, False, False, False, False, False,  True,
          True, False, False, False, False, False,  True, False, False, False,
         False,  True, False, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False, False, False, False,
         False, False, False, False, False,  True, False, False, False, False,
          True, False, False, False, False,  True, False,  True, False, False,
          True, False, False,  True, False, False, False, False,  True, False,
         False,  True, False, False, False, False, False,  True, False, False,
         False, False, False, False, False, False,  True, False, False, False,
         False, False, False, False, False,  True,  True, False, False, False,
         False, False, False,  True,  True,  True, False, False, False, False,
         False, False, False,  True, False, False, F

{'input_ids': tensor([[   2,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
          2397, 1384, 1426, 1596, 2274,    4,    4,    4,    4,    4,    4,    4,
             4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4, 2406,
          1418, 1564, 2146,  379,    4,    4,    4,    4,    4,    4, 1932, 3620,
          2178,  507, 2013, 3942, 3466,    4,    4,    4,    4,    4,    4,    4,
             4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
             4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
             4,    4,    4,    4,    4,    4,    4, 2450, 1593, 2261,    4,    4,
             4,    4,    4,    4,   41,  151,  590,    4,    4,    4,    4,    4,
             4,    4,  793,    4,    4,    4,    4,    4,    4,    4,    4,  503,
          1997,    4,    4,    4,    4,    4,    4, 1114,  347, 1374, 1386, 1434,
          1625, 2392, 1361, 1334,    4,    4,    4,    4,    4,    4, 2391,    4,
   

In [4]:
# Initializing a RoBERTa configuration
configuration = RobertaConfig(vocab_size = tokenizer.vocab_size,
                            pad_token_id = tokenizer.pad_token_id,
                            eos_token_id = tokenizer.sep_token_id,
                            bos_token_id = tokenizer.cls_token_id,
                            type_vocab_size = 1,
                            layer_norm_eps = 1e-05,
                            max_position_embeddings = 514 )
# Initializing a model from the configuration
model = RobertaForMaskedLM(configuration)

In [ ]:
model.eval()

In [10]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    num_train_epochs=10,
    do_train=True,
    per_gpu_train_batch_size=2,
    save_steps=500,
    save_total_limit=2
)
trainer = Trainer(model = model, 
                args = training_args, 
                train_dataset=train_data, 
                eval_dataset=valid_data,
                data_collator=data_collator)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.